# View and Reshape Operation in Depth 

Major difference between View and Reshape, both used to change the shape of tensors...

Transform the shape keeping data consistent. Both can change the shape, but behaves differently at the memory level

(different behavior on te underline data structure)

- view: work only on contiguous data tensor (tensor stored contiguously in memory)
- reshape: can handle non contiguous tensor data structure, by creating a new one if needed

reshape is more flexible, while view is constrained on contiguous


### What we mean with contiguou data/tensors ? 

From an original tensor [[1, 2, 3], [4, 5, 6]] 2x3 stored in memory as a sequence block (left-to-right) as 1,2,3,4,5,6... 

stored as a flat array in memory (flat list) in this case it is contiguous. 
contiguous when grid-like structure directly map to the flat memoery in row-major order (left to right and with row index)

Grid like structure directly map to flat memory. It respect the order of the original tensor in the data spac.



While if from an original data [[1, 2, 3], [4, 5, 6]], when any tensor modification change it order (e.g. transpose, slicing, etc)

it get new size [[1, 4], [2, 5], [3, 6]] New value after transposition

In memory the value is still stored as [1, 2, 3, 4, 5, 6] but it in pytorch data it is sequenced in row major order as 1 4 2 5 3 6 ordering...


Now data is sequenced in a differnet way from the memory ordering.

This make the tensor non contiguous: The data remains the same in memory, but the access pattern changes. When indexing it become computationally expensive


When working only with contiguous data, we make the code more efficient, better performing (faster) by working with view()


Instead with reshape() is less performing since non contiguous is accepted 


We can only use view when we are confident about contiguity of tensor, and need efficient operation in our system. 

Reshape can be used when we want robustness and we are unsure about contuguity. It can work also with non-contiguous. 


Reshape handles non-contiguous data by creating a contiguous copy of it. (If it take a contiguous one, it does reshape directly)

In [1]:
import torch

In [ ]:
tensor = torch.arange(12)

# try to reshape with both methods: 

reshaped_view = tensor.view(2, 6)
reshaped_reshape = tensor.reshape(2, 6)

print("Original : \n", tensor)

print("\nView : \n", reshaped_view)

print("\nReshape : \n", reshaped_reshape)

# Same information stored, of new shape

Original : 
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

View : 
 tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])

Reshape : 
 tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [4]:
# Check if contiguous tensors 
tensor.is_contiguous()
# If non-contiguous view() will give an erro while reshape() can run 

True

In [5]:
# It is possible to calculate the shape automatically of reshaping 

tensor.view(3, -1) # The -1 value indicate to automatically infer the shape based on the original tensor
# In this case we specify the number of new rows, while the other diemension is automatically computed 

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [6]:
# In CNN after some operations we use vectors...
# Given matrix MxN to get 1 x (M) vector we can use -1 to change 3D/2D tensor to 1D vector 

tensor3d = torch.arange(24).reshape(2, 3, 4) # from a 2x3x4 tensor

print(tensor3d)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])


In [ ]:
# I want to use this tensor3d as input for a CNN feature ?? 
# To create 1D tensor from this 3D
flattened = tensor3d.view(-1) # specifying -1 it automatically compute the new size 

print(flattened)
print(flattened.is_contiguous()) # Still keeping contiguity after reshaping

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23])
True


In [ ]:
tensor3d = torch.arange(24).reshape(12,2) # from a 2x3x4 tensor

# transpose it to lose contiguity 
transpose3d = tensor3d.t()

print(transpose3d.is_contiguous())

False


In [11]:
# the transposed tensor is not contiguous... 
transpose3d.view(6, 4) # try to reshape with view, BUT IT IS NON-CONTUGUOUS

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [14]:
# It suggest to use reshape instead...
new = transpose3d.reshape(6, 4) 

In [15]:
print(new.is_contiguous())

True


In [ ]:
# By reshape a contiguous copy is created... 
# view() works only on contiguous memory and cannot handle non-contiguous...
# It is possible to make data contiguous

transpose3d = transpose3d.contiguous()
transpose3d.view(6, 4) 

# Now it is create a contiguous version of it where view() can be used 

tensor([[ 0,  2,  4,  6],
        [ 8, 10, 12, 14],
        [16, 18, 20, 22],
        [ 1,  3,  5,  7],
        [ 9, 11, 13, 15],
        [17, 19, 21, 23]])

In [19]:
# In conclusion, view is more efficient and fast, but not robust
# reshape instead degrade performances by creating copies of the tensor to handle non-contiguity...